**Terminale NSI**
<div class="bg-info"><h1>Chapitre 13 - Recherche textuelle</h1></div>

# 1. Recherche naïve
![image](img/gif_naive.gif)

## 1.1 Premier algorithme

In [1]:
def recherche_naive(texte, motif):
    '''
    renvoie la liste des indices (éventuellement vide) des occurrences de
    de la chaîne motif dans la chaîne texte.
    '''
    indices = []
    i = 0
    while i <= len(texte) - len(motif):
        k = 0
        while k < len(motif) and texte[i+k] == motif[k]:
            k += 1
        if k == len(motif):
            indices.append(i)
        i += 1
    return indices

In [2]:
print(recherche_naive("une magnifique maison bleue", "maison")) # -> 15
print(recherche_naive("une magnifique maison bleue", "nsi")) # -> []
print(recherche_naive("une magnifique maison bleue", "ma")) # -> [4, 15]

[15]
[]
[4, 15]


## 1.2 Modification de l'algorithme
## Exercice 1
Re-écrire l'algorithme précédent en s'arrêtant dès qu'une occurrence de motif est trouvée dans texte.  
La fonction renverra uniquement un booléen.    

## 1.3 Application à la recherche d'un motif dans un roman
Le Projet Gutenberg permet de télécharger légalement des ouvrages libres de droits dans différents formats.  
Nous allons travailler avec le Tome 1 du roman Les Misérables de Victor Hugo au format txt.  
## 1.3.1 Récupération du texte dans une seule chaîne de caractères
```Python
with open("Les_Miserables.txt") as f:
    roman = f.read().replace('\n', ' ')
```
## 1.3.2 Vérification et mesure du temps de recherche
## Exercice 2
A l'aide du module `time`, mesurer le temps de recherche dans Les Misérables d'un mot court, d'une longue phrase (présente dans le texte), d'un mot qui n'existe pas. Que remarquez-vous ?      

In [ ]:
with open("img/Les_Miserables.txt") as f:
    roman = f.read().replace('\n', ' ')

# 2. Vers l'algorithme de Boyer-Moore : et si on partait à l'envers ?
## Exercice 3
Re-écrire l'algorithme de recherche naïve mais en démarrant de la fin du motif et non du début. Certes, c'est pour l'instant très artificiel, mais c'est un outil qui nous servira plus tard.

In [ ]:
def reverse(texte, motif):
    indices = []
    i = 0
    while i <= len(texte) - len(motif):
        k = len(motif)-1
        while k >= 0 and texte[i+k] == motif[k]:
            k -= 1
        if k == -1:
            indices.append(i)
        i += 1
    return indices

# 3. Algorithme de Boyer-Moore-Horspool
## 3.1 Principe
L'idée est d'améliorer le code précédent (celui où l'on parcourt le motif à l'envers) en **sautant** directement au prochain endroit potentiellement valide.

Pour cela on regarde le caractère X du texte sur lequel on s'est arrêté (car X n'était pas égal au caractère de rang équivalent dans le motif):

+ si X n'est pas dans le motif, il est inutile de se déplacer "de 1" : on retomberait tout de suite sur X, c'est du temps perdu. On se décale donc juste assez pour dépasser X, donc de la longueur du motif cherché.  
+ si X est dans le motif (sauf à la dernière place du motif !), on va regarder la place de la dernière occurence de X dans le motif et de déplacer de ce nombre, afin de faire coïncider le X du motif et le X du texte.  

![image](img/gif_BM.gif)  

## 3.2 Implémentation
On va d'abord coder une fonction `dico_lettres` qui renvoie un dictionnaire associant à chaque lettre de mot (paramètre d'entrée) son dernier rang dans le mot. On exclut la dernière lettre, qui poserait un problème lors du décalage (on décalerait de 0...)  
## Algorithme de Boyer-Moore-Horspool 

In [6]:
def dico_lettres(mot):
    d = {}
    for i in range(len(mot)-1):
        d[mot[i]] = i
    return d

def BMH(texte, motif):
    dico = dico_lettres(motif)
    indices = []
    i = 0
    while i <= len(texte) - len(motif):
        k = len(motif)-1
        while k >= 0 and texte[i+k] == motif[k]: # On remonte le motif à l'envers, tant qu'il y a correspondance et qu'on n'est pas arrivés au début du motif
            k -= 1
        if k == -1: # Si on est arrivés à la valeur k=-1, c'est qu'on a parcouru tout le mot : on l'a donc trouvé.
            indices.append(i)
            i += 1 # On a trouvé le motif, mais attention, il ne faut pas trop se décaler sinon on pourrait rater d'autres occurences du motif (pensez à la recherche du motif «mama» dans le mot «mamamamama»). On se décale donc de 1.
        else:
            if texte[i+k] in dico: # On s'est arrêté avant la fin, sur une lettre présente dans le mot : il va falloir faire un décalage intelligent.
                i += len(motif) - dico[texte[i+k]] - 1 # On décale juste de ce qu'il faut pour mettre en correspondance les lettres.
            else:
                i += len(motif) # La lettre n'est pas dans le motif : on se positionne juste après elle en se décalant de toute la longueur du motif.

    return indices

In [7]:
print(BMH("une magnifique maison bleue", "maison")) # ->[15]
print(BMH("une magnifique maison bleue", "nsi")) # ->[]
print(BMH("une magnifique maison bleue", "ma")) # ->[4, 15]
print(BMH("mamamamama","mama")) # ->[0, 2, 4, 6]

[15]
[]
[4, 15]
[0, 2, 4, 6]


## Exercice 4
Reprendre les mesures effectuées sur Les Misérables, mais cette fois avec l'algorithme BMH. Que remarquez-vous ?